In [1]:
import os
import sys
import PIL
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from object_detection_ign.satellite_view import WMTSClient, SatelliteView

In [2]:
# WMTS_SERVICE_URL = "https://wxs.ign.fr/satellite/geoportail/wmts"
WMTS_SERVICE_URL = "https://wxs.ign.fr/ortho/geoportail/wmts?SERVICE=WMTS"
CORRESPONDANCE_TABLE_URL = "https://developers.arcgis.com/documentation/mapping-apis-and-services/reference/zoom-levels-and-scale/"
DATA_PATH = os.path.join("..","data")
CORRESPONDANCE_TABLE_PATH = os.path.join(DATA_PATH, "correspondance_table.csv")

In [3]:
client = WMTSClient(
    WMTS_SERVICE_URL, CORRESPONDANCE_TABLE_PATH, CORRESPONDANCE_TABLE_URL
)

print(client.list_available_zoom_options())
print(client.list_available_layers())
# satellite_view = client.create_satellite_view_from_address(
#     "Aérodrome de Carpiquet", "HR.ORTHOIMAGERY.ORTHOPHOTOS", 18)
satellite_view = client.create_satellite_view_from_position(
    49.006174, 2.585743, "HR.ORTHOIMAGERY.ORTHOPHOTOS", 17)
satellite_view.show_image()

[I 221123 16:18:34 satellite_view:35] Existing correspondance table found, loading it now.


['0 - 591657527.591555 - Global', '1 - 295828763.795777 - Global', '2 - 147914381.897889 - Subcontinent', '3 - 73957190.948944 - Subcontinent', '4 - 36978595.474472 - Subcontinent', '5 - 18489297.737236 - Large country', '6 - 9244648.868618 - Large country', '7 - 4622324.434309 - Small country/US state', '8 - 2311162.217155 - Small country/US state', '9 - 1155581.108577 - Large metropolitan area', '10 - 577790.554289 - Large metropolitan area', '11 - 288895.277144 - City', '12 - 144447.638572 - Town', '13 - 72223.819286 - Village', '14 - 36111.909643 - Village', '15 - 18055.954822 - Small road', '16 - 9027.977411 - Street', '17 - 4513.988705 - Street block', '18 - 2256.994353 - Address', '19 - 1128.497176 - Street intersection', '20 - 564.248588 - Street intersection', '21 - 282.124294 - Street intersection']
dict_keys(['HR.ORTHOIMAGERY.ORTHOPHOTOS', 'ORTHOIMAGERY.ORTHOPHOS.RESTRICTEDAREAS', 'ORTHOIMAGERY.ORTHOPHOTOS', 'ORTHOIMAGERY.ORTHOPHOTOS.BDORTHO', 'ORTHOIMAGERY.ORTHOPHOTOS.COAST

100%|██████████| 3/3 [00:01<00:00,  1.95it/s]


In [ ]:
satellite_view.image.save(os.path.join(DATA_PATH, "test_img.jpg"))

In [6]:
# Load model

import onnx
from onnx_tf.backend import prepare

model = onnx.load(os.path.join('..', 'models', 'model_export_v3.onnx'))

prepare(model).export_graph('model')

c:\Users\berto\mambaforge\envs\satellite\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


BackendIsNotSupposedToImplementIt: in user code:

    File "c:\Users\berto\mambaforge\envs\satellite\lib\site-packages\onnx_tf\backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "c:\Users\berto\mambaforge\envs\satellite\lib\site-packages\onnx_tf\backend.py", line 347, in _onnx_node_to_tensorflow_op  *
        return handler.handle(node, tensor_dict=tensor_dict, strict=strict)
    File "c:\Users\berto\mambaforge\envs\satellite\lib\site-packages\onnx_tf\handlers\handler.py", line 61, in handle  *
        raise BackendIsNotSupposedToImplementIt("{} version {} is not implemented.".format(node.op_type, cls.SINCE_VERSION))

    BackendIsNotSupposedToImplementIt: Unsqueeze version 13 is not implemented.
